In [10]:
import pandas as pd

# Exploration #

In [11]:
df = pd.read_csv("https://raw.githubusercontent.com/GeoDS/COVID19USFlows/master/weekly_flows/ct2ct/2021_03_02/weekly_ct2ct_2021_03_02_0.csv")

In [12]:
df.head()

,geoid_o,geoid_d,lng_o,lat_o,lng_d,lat_d,date_range,visitor_flows,pop_flows
0,1001020100,1001020100,-86.490076,32.477185,-86.490076,32.477185,03/02/21 - 03/08/21,14,158.0
1,1001020100,1001020200,-86.490076,32.477185,-86.473375,32.474248,03/02/21 - 03/08/21,44,499.0
2,1001020100,1001020300,-86.490076,32.477185,-86.460190,32.475428,03/02/21 - 03/08/21,174,1975.0
3,1001020100,1001020400,-86.490076,32.477185,-86.443624,32.472001,03/02/21 - 03/08/21,61,692.0
4,1001020100,1001020500,-86.490076,32.477185,-86.422661,32.458833,03/02/21 - 03/08/21,275,3122.0


In [13]:
df.dtypes

geoid_o            int64
geoid_d            int64
lng_o            float64
lat_o            float64
lng_d            float64
lat_d            float64
date_range        object
visitor_flows      int64
pop_flows        float64
dtype: object

In [23]:
df_nyc = df[ (df['geoid_d'].astype(str).str.startswith('36061')) | (df['geoid_o'].astype(str).str.startswith('36061') ) ]

In [24]:
df_nyc.head()

,geoid_o,geoid_d,lng_o,lat_o,lng_d,lat_d,date_range,visitor_flows,pop_flows
7329,1081040700,36061004500,-85.495071,32.595459,-73.999391,40.720570,03/02/21 - 03/08/21,4,24.0
7330,1081040700,36061004700,-85.495071,32.595459,-74.002863,40.722419,03/02/21 - 03/08/21,4,24.0
7331,1081040700,36061010000,-85.495071,32.595459,-73.971232,40.758065,03/02/21 - 03/08/21,4,24.0
7332,1081040700,36061013700,-85.495071,32.595459,-73.980234,40.765207,03/02/21 - 03/08/21,4,24.0
7823,1081040902,36061010900,-85.467959,32.620428,-73.989703,40.752250,03/02/21 - 03/08/21,4,37.0


In [25]:
df_nyc.size

1152

# Bulk loading #

In [96]:
def wkt_loads(x):
    try:
        return wkt.loads(x)
    except Exception:
        return None

In [97]:
#REQUIRE a geodataframe with a column named 'geometry_source' and another named 'geometry_target'
#                   and with an unique row-identifier column named 'id'
#REQUIRE a tesselletion in OpenStreetMap format

def flow_from_flow_in_tess(df_flow_city, tess_city):
    gdf_1 = gpd.GeoDataFrame(df_flow_city, crs="EPSG:4326", geometry=df_flow_city['geometry_source']).drop(['geometry_target', 'geometry_source'],axis =1)
    
    res_merge_source = gpd.sjoin(tess_city, gdf_1, how='right', op='contains')
    res_merge_source.dropna(inplace=True)
    res_merge_source.reset_index(drop=True, inplace=True)
    
    print("res_merge_source")
    print(res_merge_source.head())
    
    gdf_2 = gpd.GeoDataFrame(df_flow_city, crs="EPSG:4326", geometry=df_flow_city['geometry_target']).drop(['geometry_target', 'geometry_source'],axis =1)

    print(gdf_2.head())
    
    res_merge_dest = gpd.sjoin(tess_city, gdf_2, how='right', op='contains')
    res_merge_dest.dropna(inplace=True)
    res_merge_dest.reset_index(drop=True,inplace=True)
    
    print("res_merge_dest")
    print(res_merge_dest.head())
    
    fdf = res_merge_dest.merge(res_merge_source, how='inner', on=['id'])
    fdf = fdf[['tile_ID_x','tile_ID_y','pop_flows']]
    
    print("fdf")
    print(fdf.head())
    
    flusso = fdf.groupby(["tile_ID_x", "tile_ID_y"]).size().reset_index(name="flow") #here must be a sum of the flows, not a size()
    flusso = flusso.rename(columns={'tile_ID_x': 'origin', 'tile_ID_y':'destination'})
    
    print(flusso.head())
    
    fdf_flow_city = skmob.FlowDataFrame(data = flusso,tessellation=tess_city, tile_id='tile_ID', origin ='origin', 
                              destination = 'destination', flow = 'flow')
    
    return fdf_flow_city
    

In [98]:
def to_Adj_Matrix(df):
    df_np = df.to_numpy().astype(int)
    n = max(max(origin, destination) for origin, destination, flow in df_np )
    
    matrix = np.zeros((n+1, n+1))
    for origin, destination, flow in df_np:
        matrix[origin][destination] = flow 
    return matrix

In [99]:
def pad_with_zeros(A, r, c):
   out = np.zeros((r, c))
   r_, c_ = np.shape(A)
   out[0:r_, 0:c_] = A
   return out

In [5]:
! time python ./codes/download_weekly_data.py --start_year 2021 --start_month 2 --start_day 1  --end_day 8   --output_folder ./data/weekly_flows   --ct 


real	0m40,488s
user	0m15,767s
sys	0m2,295s


In [8]:
! python ./codes/merge_files.py -i ./data/weekly_flows/ct2ct/2021_02_01  -o ./data/weekly_flows/ct2ct/2021_02_01.csv

In [12]:
df_usa = pd.read_csv("./data/weekly_flows/ct2ct/2021_02_01.csv")

In [13]:
df_usa.head()

,geoid_o,geoid_d,lng_o,lat_o,lng_d,lat_d,date_range,visitor_flows,pop_flows
0,47037013202,48037010800,-86.871483,36.158244,-94.086227,33.436190,02/01/21 - 02/07/21,4,135.0
1,47037013202,47037018201,-86.871483,36.158244,-86.891380,36.115338,02/01/21 - 02/07/21,155,5261.0
2,47037013202,47187050504,-86.871483,36.158244,-87.124490,35.956061,02/01/21 - 02/07/21,4,135.0
3,47037013202,47037018404,-86.871483,36.158244,-86.894913,36.067961,02/01/21 - 02/07/21,4,135.0
4,47037013202,47037016700,-86.871483,36.158244,-86.830043,36.132917,02/01/21 - 02/07/21,8,271.0


In [14]:
df_usa.size

59051808

# NYC #

In [102]:
df_nyc = df_usa[ (df_usa['geoid_d'].astype(str).str.startswith('36061')) &
                (df_usa['geoid_o'].astype(str).str.startswith('36061') ) ]

In [103]:
df_nyc.head()

,geoid_o,geoid_d,lng_o,lat_o,lng_d,lat_d,date_range,visitor_flows,pop_flows
92933,36061027300,36061014500,-73.937385,40.853979,-73.984255,40.769809,02/01/21 - 02/07/21,4,290.0
92934,36061027300,36061014300,-73.937385,40.853979,-73.965554,40.782477,02/01/21 - 02/07/21,4,290.0
92936,36061027300,36061007400,-73.937385,40.853979,-73.984083,40.746214,02/01/21 - 02/07/21,4,290.0
92938,36061027300,36061002100,-73.937385,40.853979,-74.010043,40.714330,02/01/21 - 02/07/21,4,290.0
92941,36061027300,36061018700,-73.937385,40.853979,-73.975041,40.799329,02/01/21 - 02/07/21,4,290.0


In [104]:
df_nyc.size

85437

In [105]:
from shapely.geometry import Point, Polygon
from shapely import wkt
import skmob
from skmob.preprocessing import filtering, detection
from skmob.utils.plot import plot_gdf
from skmob.tessellation import tilers
import pandas as pd
import geopandas as gpd
import webbrowser
import numpy as np
import networkx as nx

pd.set_option('display.max_columns', 500)

In [106]:
meters = 500

In [107]:
def wkt_loads(x):
    try:
        return wkt.loads(x)
    except Exception:
        return None

In [108]:
tess_nyc = tilers.tiler.get("squared", meters=meters, base_shape="New York City, New York")

/home/giovanni/anaconda3/envs/skmob/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [109]:
df_nyc['id'] = df_nyc.apply(lambda x: hash(tuple(x)), axis = 1)

/home/giovanni/anaconda3/envs/skmob/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [110]:
df_nyc['geometry_source'] = [Point(xy) for xy in zip(df_nyc['lng_o'],df_nyc['lat_o'])]
df_nyc['geometry_target'] = [Point(xy) for xy in zip(df_nyc['lng_d'],df_nyc['lng_d'])]


/home/giovanni/anaconda3/envs/skmob/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/giovanni/anaconda3/envs/skmob/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [111]:
df_nyc.head()

,geoid_o,geoid_d,lng_o,lat_o,lng_d,lat_d,date_range,visitor_flows,pop_flows,id,geometry_source,geometry_target
92933,36061027300,36061014500,-73.937385,40.853979,-73.984255,40.769809,02/01/21 - 02/07/21,4,290.0,8828090250841860075,POINT (-73.93738465130313 40.85397911228601),POINT (-73.98425504939362 -73.98425504939362)
92934,36061027300,36061014300,-73.937385,40.853979,-73.965554,40.782477,02/01/21 - 02/07/21,4,290.0,-2922195811220374125,POINT (-73.93738465130313 40.85397911228601),POINT (-73.96555390421362 -73.96555390421362)
92936,36061027300,36061007400,-73.937385,40.853979,-73.984083,40.746214,02/01/21 - 02/07/21,4,290.0,6439931955471095023,POINT (-73.93738465130313 40.85397911228601),POINT (-73.98408286389029 -73.98408286389029)
92938,36061027300,36061002100,-73.937385,40.853979,-74.010043,40.714330,02/01/21 - 02/07/21,4,290.0,-5709007629091266646,POINT (-73.93738465130313 40.85397911228601),POINT (-74.01004337525612 -74.01004337525612)
92941,36061027300,36061018700,-73.937385,40.853979,-73.975041,40.799329,02/01/21 - 02/07/21,4,290.0,-5304612784201713693,POINT (-73.93738465130313 40.85397911228601),POINT (-73.97504119505521 -73.97504119505521)


In [112]:
flow_nyc = df_nyc

flow_nyc = flow_nyc[['id','geoid_o', 'geoid_d','pop_flows', 'geometry_source', 'geometry_target']]

flow_nyc.head()

,id,geoid_o,geoid_d,pop_flows,geometry_source,geometry_target
92933,8828090250841860075,36061027300,36061014500,290.0,POINT (-73.93738465130313 40.85397911228601),POINT (-73.98425504939362 -73.98425504939362)
92934,-2922195811220374125,36061027300,36061014300,290.0,POINT (-73.93738465130313 40.85397911228601),POINT (-73.96555390421362 -73.96555390421362)
92936,6439931955471095023,36061027300,36061007400,290.0,POINT (-73.93738465130313 40.85397911228601),POINT (-73.98408286389029 -73.98408286389029)
92938,-5709007629091266646,36061027300,36061002100,290.0,POINT (-73.93738465130313 40.85397911228601),POINT (-74.01004337525612 -74.01004337525612)
92941,-5304612784201713693,36061027300,36061018700,290.0,POINT (-73.93738465130313 40.85397911228601),POINT (-73.97504119505521 -73.97504119505521)


In [113]:
flow_nyc.dtypes

id                   int64
geoid_o              int64
geoid_d              int64
pop_flows          float64
geometry_source     object
geometry_target     object
dtype: object

In [114]:
flow_nyc = flow_from_flow_in_tess(flow_nyc, tess_nyc)

/home/giovanni/anaconda3/envs/skmob/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  


res_merge_source
   index_left tile_ID                   id      geoid_o      geoid_d  \
0        3888    3888  8828090250841860075  36061027300  36061014500   
1        3888    3888 -2922195811220374125  36061027300  36061014300   
2        3888    3888  6439931955471095023  36061027300  36061007400   
3        3888    3888 -5709007629091266646  36061027300  36061002100   
4        3888    3888 -5304612784201713693  36061027300  36061018700   

   pop_flows                    geometry  
0      290.0  POINT (-73.93738 40.85398)  
1      290.0  POINT (-73.93738 40.85398)  
2      290.0  POINT (-73.93738 40.85398)  
3      290.0  POINT (-73.93738 40.85398)  
4      290.0  POINT (-73.93738 40.85398)  
                        id      geoid_o      geoid_d  pop_flows  \
92933  8828090250841860075  36061027300  36061014500      290.0   
92934 -2922195811220374125  36061027300  36061014300      290.0   
92936  6439931955471095023  36061027300  36061007400      290.0   
92938 -57090076290912666

/home/giovanni/anaconda3/envs/skmob/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326



res_merge_dest
Empty GeoDataFrame
Columns: [index_left, tile_ID, id, geoid_o, geoid_d, pop_flows, geometry]
Index: []


KeyError: "['pop_flows'] not in index"